In [ ]:
dataset_root = "../dataset/"
images_dir = dataset_root + "complete/"
features_dir = dataset_root + 'handcrafted/'

recompute_features = False
classify_images = True

In [ ]:
import sys
import cv2 as cv
import matplotlib.pyplot as plt
import os
from tensorflow import keras
import pandas as pd
sys.path.append("..")

from keras.applications.efficientnet_v2 import EfficientNetV2B0, preprocess_input
from feature_extraction import build_feature_extractor, cut_layer_of_model, unroll_layers
from similarity_search.neural_similarity_search import extract_features, order_prediction
from handcrafted_extraction import load_all_features, load_features

model = keras.models.load_model('../classification/tuned_models/efficientnet_v2_cosine')

def builder():
    model = keras.models.load_model('../classification/tuned_models/efficientnet_v2_cosine')
    return keras.Sequential(model.layers[:-1])
    #return build_feature_extractor(EfficientNetV2B0, 'top_dropout')

In [ ]:
batch_size = 128
image_size = (224, 224)

if recompute_features:
    extracted_features, filenames = extract_features(builder, images_dir, batch_size=batch_size, image_size=image_size, rebuild_interval=50, preprocessing=preprocess_input)
    os.makedirs(extracted_features_root, exist_ok=True)
    names_df = pd.DataFrame({'filename': filenames})
    names_df.to_csv(filenames_path, header=None)
    np.save(extracted_features_path, extracted_features)

In [ ]:
if recompute_features:
    

## Classify images for similarity search

In [ ]:
model = keras.models.load_model('../classification/tuned_models/efficientnet_v2_cosine')

In [ ]:
from misc import load, load_class_labels
from data_loader import data_loader
from tqdm import tqdm

dataset_root = "../dataset/"
labels_path = dataset_root + 'labels/'

complete_dir = dataset_root + "complete/"

train_info = load(labels_path + 'train.csv', 1, 1)
val_info = load(labels_path + 'validation.csv')
test_info = load(labels_path + 'test.csv')
all_info = pd.concat([train_info, val_info], axis=0)
all_info = pd.concat([all_info, test_info], axis=0)

dl = data_loader(all_info, complete_dir, 400, (224, 224));

In [ ]:
all_filenames = []
all_labels = []
all_confidence = []

significance_threshold = 0.9

if classify_images:
    for batch in tqdm(range(0, dl.number_of_batch())):
        images, _, filenames = dl.get_batch(batch, preprocessing=preprocess_input);
        predictions = model.predict(images, verbose=0)
        for filename, prediction in zip(filenames, predictions):
            prediction, ordered_classes = order_prediction(prediction, significance_threshold)

            all_filenames += [filename]
            all_confidence += [prediction]
            all_labels += [ordered_classes]

    df = pd.DataFrame({'filenames': all_filenames, 'labels': np.array(all_labels), 'confidence': np.array(all_confidence)})
    df.to_pickle('./extracted_features/predictions.pickle')
